Walk through the available list of calibration images, and extract rvec/tvec info from each one.

Write results to a file.

Intended to be used to for building the ArucoBall Graph.

In [51]:
import os
import cv2
from aruco_classes import ArucoBall, ArucoMarker, get_markers_in_single_frame
from utils import rotationMatrixToEulerAngles, rad_to_deg, calculate_net_rotation
import numpy as np

In [52]:
CALIB_IMAGES_DIR = 'calib/from_video'
OUTPUT_FILE_PATH = CALIB_IMAGES_DIR+'/discovered_markers.txt'

READ_VECTORS_FROM_FILE = 0

In [53]:
# either read previously computed vectors
if READ_VECTORS_FROM_FILE:
    id_rvec_tvecs_list = eval(open(OUTPUT_FILE_PATH, 'r').read())
# or compute them from scratch using image files
else:
    # for each image file in this directory:
    image_filenames = [f for f in os.listdir(CALIB_IMAGES_DIR) if os.path.isfile(os.path.join(CALIB_IMAGES_DIR, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    id_rvec_tvecs_list = []
    # output file
    for image_file in sorted(image_filenames):
        # Process each image frame:
        print(f'Processing {image_file}')
        im = cv2.imread(os.path.join(CALIB_IMAGES_DIR, image_file))

        # extract rvecs and tvecs for markers
        id_list, rvec_list, tvec_list = get_markers_in_single_frame(im) 
        id_list = [id[0] for id in id_list]

        # format rvecs and tvecs for file writing
        rvec_list = [rvec[0] for rvec in rvec_list]
        rvec_list = [','.join(map(str,map(lambda x:str(x[0]), rvec))) for rvec in rvec_list]
        rvec_list = [list(map(float,eval(x)))for x in rvec_list]
        
        tvec_list = [rvec[0] for rvec in tvec_list]
        tvec_list = [','.join(map(str,map(lambda x:str(x[0]), rvec))) for rvec in tvec_list]
        tvec_list = [list(map(float,eval(x)))for x in tvec_list]
        # tvec_list = [tvec for tvec in tvec_list]
        
        id_rvec_tvecs_list.append([id_list, rvec_list, tvec_list])

Processing im01.png
Processing im02.png
Processing im03.png
Processing im04.png
Processing im05.png
Processing im06.png
Processing im07.png
Processing im08.png
Processing im09.png
Processing im10.png
Processing im11.png
Processing im12.png
Processing im13.png
Processing im14.png


In [54]:
id_rvec_tvecs_list

[[[18, 11, 41, 27, 34, 25, 26, 5, 15],
  [[1.2258230701761679, 2.1789833139768033, 0.26578763646280007],
   [-2.7732995001466167, 0.10369043750725825, -0.3723345690505771],
   [-1.543050802031652, 1.7812716711286358, -0.4635010456453828],
   [0.4609759330814954, 2.612216225324735, -0.7987816940814819],
   [-2.4261587561934195, 0.027559855599630735, 0.20557811955023853],
   [-0.8146956868470753, -2.3358583169534173, 0.3165566742755903],
   [-0.6365358997424693, -2.961726457748298, -0.005782203408804371],
   [-0.30205993811549003, 2.015670693202783, 0.5384625364129514],
   [-2.5880223288140534, -0.49370417764368596, 0.521566545945236]],
  [[-0.04270977252936652, -0.027665652274264253, 0.369107064101438],
   [-0.08182460111602687, 0.04842730305181452, 0.39025066423620397],
   [-0.02520318677203341, 0.023161333933151736, 0.3844495703900895],
   [-0.07831005054060189, -0.08518652553349668, 0.3979966521240609],
   [-0.1285574599091957, 0.09034078487583397, 0.42016820993204396],
   [-0.186005

In [55]:
if not READ_VECTORS_FROM_FILE:
    # write vectors to file if re-computed
    with open(OUTPUT_FILE_PATH, 'w') as f:
        f.write('['+',\n'.join(map(str,id_rvec_tvecs_list))+']')

In [56]:
# def build_graph(id_rvec_tvecs_list):
G = ArucoBall()
for frame_info in id_rvec_tvecs_list:
    id_list, rvec_list, tvec_list = frame_info
    G.add_adjacencies_from_frame(frame_info, verbose=False)

In [57]:
out_graph_path = OUTPUT_FILE_PATH.replace('.txt', '.gexf')
G.export_graph(out_graph_path)

In [84]:
PRIMARY = 25
# set PRIMARY as the marker from which to get other markers' orientations
G.setPrimaryMarkerID(PRIMARY, verbose=True)

Primary Marker ID set to 28.


In [85]:
in_network = sorted(list(G.network.nodes))
print('NODES IN THE NETWORK:\n', f', '.join(map(str,in_network)))
# rand_node_in_network = np.random.choice(in_network)
rand_node_in_network = 25

NODES IN THE NETWORK:
 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 109, 216


In [86]:
len(G.network.nodes)

44

In [87]:
print(F'finding a path from {rand_node_in_network} to {PRIMARY}:')
path_to_prim = G.getPathToPrimaryMarker(rand_node_in_network)
path_to_prim

finding a path from 25 to 28:


[25, 28]

In [88]:
w = G.get_weights_on_path(path_to_prim)
print(f'transforms required to get from marker {rand_node_in_network} to marker {PRIMARY} are:\n{w}')

transforms required to get from marker 25 to marker 28 are:
[array([[ 0.77654097, -0.19891532, -0.59784347],
       [-0.33069551,  0.67898466, -0.65545428],
       [ 0.53630644,  0.70669126,  0.461479  ]])]


In [89]:
mat = G.get_rotation_to_primary(rand_node_in_network)
mat

array([[ 0.77654097, -0.19891532, -0.59784347],
       [-0.33069551,  0.67898466, -0.65545428],
       [ 0.53630644,  0.70669126,  0.461479  ]])

In [90]:
net_rot = calculate_net_rotation(w)
net_rot

array([[ 0.77654097, -0.19891532, -0.59784347],
       [-0.33069551,  0.67898466, -0.65545428],
       [ 0.53630644,  0.70669126,  0.461479  ]])

In [91]:
print(f'net degree rotation required to get from marker {rand_node_in_network} to marker {PRIMARY}:\n{rad_to_deg(rotationMatrixToEulerAngles(net_rot))}')

net degree rotation required to get from marker 25 to marker 28:
[56.85491693254171, -32.43255429947788, -23.067075786172374]


In [98]:
temp = G.getRotationMatrixToPrimary(rand_node_in_network)
temp

array([[ 0.77654097, -0.19891532, -0.59784347],
       [-0.33069551,  0.67898466, -0.65545428],
       [ 0.53630644,  0.70669126,  0.461479  ]])

In [99]:
G.network[rand_node_in_network][PRIMARY]['weight']

array([[ 0.77654097, -0.19891532, -0.59784347],
       [-0.33069551,  0.67898466, -0.65545428],
       [ 0.53630644,  0.70669126,  0.461479  ]])

In [100]:
G.network[PRIMARY]

AtlasView({27: {'weight': array([[ 0.89531465,  0.0655035 , -0.44059161],
       [ 0.20940786,  0.81111024,  0.54612134],
       [ 0.39314123, -0.58121378,  0.71248194]]), 'label': '28-27'}, 10: {'weight': array([[-0.63618019,  0.00801274, -0.7714989 ],
       [-0.42268905, -0.84015061,  0.33982484],
       [-0.64545235,  0.54229396,  0.53787408]]), 'label': '28-10'}, 35: {'weight': array([[ 0.85253845,  0.38923122, -0.34882266],
       [-0.52174051,  0.67343486, -0.52371016],
       [ 0.031065  ,  0.62847796,  0.77720681]]), 'label': '28-35'}, 14: {'weight': array([[-0.62748767, -0.72029693, -0.29568828],
       [ 0.7725972 , -0.62315337, -0.12154609],
       [-0.09670987, -0.3047166 ,  0.94752044]]), 'label': '28-14'}, 6: {'weight': array([[ 0.77927388, -0.62656198,  0.01234109],
       [ 0.56348773,  0.69193679, -0.45133696],
       [ 0.27425132,  0.35866916,  0.89226826]]), 'label': '28-6'}, 25: {'weight': array([[ 0.77654097, -0.33069551,  0.53630644],
       [-0.19891532,  0.6789

In [101]:
G.network[rand_node_in_network]

AtlasView({18: {'weight': array([[ 0.54397928,  0.14045198, -0.82726041],
       [ 0.63523686,  0.57521115,  0.51537004],
       [ 0.54823415, -0.80585692,  0.22368267]]), 'label': '25-18'}, 11: {'weight': array([[-0.53175071, -0.79096487, -0.30268095],
       [ 0.84349664, -0.52664646, -0.10562636],
       [-0.07585911, -0.31147725,  0.94722084]]), 'label': '25-11'}, 41: {'weight': array([[-0.72724923,  0.22489928, -0.64848197],
       [-0.64001557,  0.11913149,  0.75907032],
       [ 0.24796899,  0.96707187,  0.05730076]]), 'label': '25-41'}, 27: {'weight': array([[ 0.44312556,  0.0173718 , -0.89629122],
       [-0.08250002,  0.99635962, -0.02147662],
       [ 0.8926553 ,  0.08346088,  0.44294559]]), 'label': '25-27'}, 34: {'weight': array([[-0.62965736, -0.7638866 ,  0.14145268],
       [ 0.70495228, -0.48530118,  0.51722823],
       [-0.32645656,  0.42539395,  0.84407707]]), 'label': '25-34'}, 26: {'weight': array([[ 0.74482445,  0.45662847, -0.48654597],
       [-0.38747907,  0.88

In [106]:
clique = [25, 9, 16]
mat_a_b = G.network[clique[0]][clique[1]]['weight']
mat_b_c = G.network[clique[1]][clique[2]]['weight']
mat_a_c = G.network[clique[0]][clique[2]]['weight']

In [107]:
mat_a_b, mat_b_c, mat_a_c

(array([[ 0.89771535,  0.21890796,  0.38234338],
        [-0.15796203,  0.97005303, -0.18451319],
        [-0.41128476,  0.10524459,  0.90541064]]),
 array([[-0.40698228,  0.91029421, -0.07569599],
        [-0.90331974, -0.41338977, -0.1145528 ],
        [-0.1355687 ,  0.02175673,  0.99052904]]),
 array([[-0.61493192,  0.73500931,  0.28569224],
        [-0.78696609, -0.54881632, -0.28193087],
        [-0.05042925, -0.39819839,  0.91591208]]))

In [108]:
route_1 = mat_a_b @ mat_b_c
route_2 = mat_a_c

In [109]:
route_1-route_2

array([[ 0.00000000e+00,  6.66133815e-16,  5.55111512e-17],
       [-1.11022302e-16, -5.55111512e-16,  0.00000000e+00],
       [ 0.00000000e+00, -4.44089210e-16,  0.00000000e+00]])

In [110]:
pickle_path = CALIB_IMAGES_DIR+'/arupickle.pkl'
G.save(pickle_path)

In [111]:
H = ArucoBall()
H.load(pickle_path)